# GBM

부스팅 알고리즘은 여러 개의 약한 학습기(Weak learner)를 순차적으로 학습-예측하면서 잘못 예측한 데이터에 가중치 부여를 통해 오류를 개선해 나가면서 학습하는 방식  

부스팅 알고리즘의 대표적인 예:  
- AdaBoost(Adaptive boosting)
    - 오류 데이터에 가중치를 부여하면서 부스팅을 수행함
- 그래디언트 부스트

<br>

<에이다 부스트 진행 과정 도식화>
![image](https://user-images.githubusercontent.com/70187490/128980343-655aafc7-6ad2-4005-8f4a-e5282d1e2709.png)

* Step 1은 첫 번째 약한 학습기 (weak learner)가 분류 기준1로 +와 -를 분류한 것. 동그라미로 표시된 데이터는 +데이터가 잘못 분류한 오류 데이터
* Step 2에서 오류 데이터에 대해서 가중치 값을 부여
* Step 3에서 두 번째 약한 학습기가 분류 기준 2로 +와 -를 분류. 동그라미로 표시된 -데이터가 잘못 분류된 오류 데이터
* Step 4에선 잘못 분류된 이 - 오류 데이터에 대해 다음 약한 학습기가 잘 분류할 수 있게 더 큰 가중치를 부여함
* Step 5에선 세 번째 약한 학습기가 분류 기준 3으로 +와 -를 분류하고 오류 데이터를 찾음. 에이다 부스트는 약한 학습기가 순차적으로 오류 값에 대해 가중치를 부여한 예측 결정 기준을 모두 결합해 예측을 수행
* 마지막으로 첫번째, 두번째, 세번째 약한 학습기를 모두 결합한 결과 예측.  

<가중치로 부여해 결합하는 에이다 부스트 진행 과정 도식화>
![image](https://user-images.githubusercontent.com/70187490/128989366-ced88655-286e-4651-9aae-46e950c8b6ce.png)

GBM도 이 방법과 유사하나, 가중치 업데이트를 경사하강법(Gradient Descent)를 이용함  
분류의 실제 결괏값을 $y$, 피러를 $x_1, x_2, ... ,x_n$, 그리고 피처에 기반한 예측 함수를 $F(x)$함수라 할 때,  
오류식은 $h(x)=y-F(x)$가 되고 이를 최소화하는 방향성을 가지고 반복적으로 가중치 값을 업데이트 함  
-> '반복 수행을 통해 오류를 최소화할 수 있도록 가중치의 업데이트 값을 도출하는 기법'  

<br>

GBM은 CART 기반의 알고리즘과 마찬가지로 분류, 회귀 모두 가능  
과적합에도 매우 뛰어난 성능을 가지고 있음

In [6]:
def get_new_feature_name_df(old_feature_name_df):

    feature_dup_df = pd.DataFrame(data=old_feature_name_df.groupby('column_name').cumcount(),
                                  columns=['dup_cnt'])
    feature_dup_df = feature_dup_df.reset_index()
    new_feature_name_df = pd.merge(old_feature_name_df.reset_index(), feature_dup_df, how='outer')
    new_feature_name_df['column_name'] = new_feature_name_df[['column_name', 'dup_cnt']].apply(lambda x: x[0] + '_'+str(x[1])
                                                                                              if x[1] > 0 else x[0], axis=1)
    new_feature_name_df=new_feature_name_df.drop(['index'], axis=1)
    
    return new_feature_name_df

def get_human_dataset( ):

    # 각 데이터 파일은 공백으로 분리되어 있으므로 read_csv에서 공백 문자를 sep으로 할당
    feature_name_df = pd.read_csv('D:\Python/human_activity/features.txt', sep='\s+',
                                  header=None, names=['column_index', 'column_name'])
    
    # 중복된 피처명을 수정하는 get_new_feature_name_df()를 이용, 신규 피처명 DataFrame 생성
    new_feature_name_df = get_new_feature_name_df(feature_name_df)
    
    # DataFrame에 피처명을 칼럼으로 부여하기 위해 리스트 객체로 다시 변환
    feature_name = new_feature_name_df.iloc[:,1].values.tolist()
    
    # 학습 피처 데이터세트와 테스트 피처 데이터를 DataFrame으로 로딩. 칼럼명은 feature_name 적용
    X_train = pd.read_csv('D:\Python/human_activity/train/X_train.txt', sep='\s+', names=feature_name)
    X_test = pd.read_csv('D:\Python/human_activity/test/X_test.txt', sep='\s+', names=feature_name)
    
    # 학습 레이블과 테스트 레이블 데이터를 DataFrame으로 로딩하고 칼럼명은 action으로 부여
    y_train = pd.read_csv('D:\Python/human_activity/train/y_train.txt', sep='\s+', names=['action'])
    y_test = pd.read_csv('D:\Python/human_activity/test/y_test.txt', sep='\s+', names=['action'])
    
    # 로드된 학습/테스트용 DataFrame을 모두 반환
    return X_train, X_test, y_train, y_test

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import time
import warnings
warnings.filterwarnings('ignore')

X_train, X_test, y_train, y_test = get_human_dataset( )

# GBM 수행 시간 측정을 위함. 시간 시간 설정
start_time = time.time()

gb_clf = GradientBoostingClassifier(random_state=0)
gb_clf.fit(X_train, y_train)
gb_pred = gb_clf.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)

print('GBM 정확도: {0:.4f}'.format(gb_accuracy))
print('GBM 수행 시간: {0:.1f}'.format(time.time()-start_time))

GBM 정확도: 0.9389
GBM 수행 시간: 574.4


GBM이 랜덤포레스트보다 조금 더 나은 예측 성능을 보일지라도,  
- 수행시간이 오래 걸리고
- 하이퍼 파라미터 튜닝에도 신경을 써야 함
- 사이킷런의 GradientBoostingClassifier는 약한 학습기의 순차적인 예측 오류 보정을 통해 학습을 수행하므로 병렬 처리가 지원하지 않으므로 매우 많은 학습시간이 필요

## GBM 하이퍼 파라미터 및 튜닝

- loss: 경사 하강법에서 사용할 비용함수를 지정. 특별한 이유가 없으면 기본값인 'deviance'를 그대로 적용
- learning_rate: 
    - GBM이 학습을 진행할 때마다 적용하는 학습률. Weak learner가 순차적으로 오류 값을 보정해 나가는데 적용하는 계수.
    - 0~1 사이의 값을 지정할 수 있으며 기본값은 0.1
    - 너무 작은 값을 적용하면 업데이트되는 값이 작아져서 최소 오류 값을 찾아 예측 성능이 높아질 가능성이 높음
    - 더불어 모든 weak learner의 반복이 완료되어도 최소 오류 값을 찾지 못할 수도 있음
    - 반대로 너무 큰 값을 적용하면 최소 오류 값을 찾지 못하고 그냥 지나쳐 버려 예측 성능이 떨어질 가능성이 높아지지만, 빠른 수행이 가능함
    - 이러한 특성 때문에 learning_rate는 n_estimators와 상호 보완적으로 조합해 사용함
    - learning_rate를 작게 하고 n_estimators를 크게 하면 더 이상 성능이 좋아지지 않는 한계점까지는 예측 성능이 좋아질 수 있지만
    - 수행시간이 너무 오래걸리는 단점이 있고 예측 성능 역시 현격히 좋아지지는 않음
- n_estimators: 
    - weak learner의 개수. 
    - weak learner가 순차적으로 오류를 보정하므로 개수가 많을 수록 예측 성능이 일정 수준까지는 좋아질 수 있음
    - 하지만 갯수가 많을 수록 수행시간이 오래 걸림
    - 기본 값은 100
- subsample: 
    - weak learner가 학습에 사용하는 데이터의 샘플링 비율
    - 기본 값은 1
    - 이는 전체 학습 데이터를 기반으로 학습한다는 의미(0.5이면 학습 데이터의 50%)
    - 과적합이 염려되는 경우 subsample을 1보다 작은 값으로 설정

In [15]:
from sklearn.model_selection import GridSearchCV

params = {
    'n_estimators':[100, 500],
    'learning_rate':[0.05, 0.1]
}

start_time = time.time()
grid_cv = GridSearchCV(gb_clf, param_grid=params, cv=2, verbose=1)
grid_cv.fit(X_train, y_train)

print('최적 하이퍼 파라미터: \n', grid_cv.best_params_)
print('최고 예측 정확도: {0:.4f}'.format(grid_cv.best_score_))
print('GridSearchCV 수행 시간: {0:.1f}'.format(time.time()-start_time))

Fitting 2 folds for each of 4 candidates, totalling 8 fits
최적 하이퍼 파라미터: 
 {'learning_rate': 0.1, 'n_estimators': 500}
최고 예측 정확도: 0.9011
GridSearchCV 수행 시간: 9416.5


In [16]:
# GridsearchCV를 이용해 최적으로 학습된 estimator로 예측 수행
gd_pred=grid_cv.best_estimator_.predict(X_test)
gb_accuracy = accuracy_score(y_test, gb_pred)
print('GBM 정확도: {0:.4f}'.format(gb_accuracy))

GBM 정확도: 0.9389
